In [ ]:
# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function

from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
# import argparse
import os
import pprint

In [ ]:
# reparameterization trick
# instead of sampling from Q(z|X), sample epsilon = N(0,I)
# z = z_mean + sqrt(var) * epsilon
def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.

    # Arguments
        args (tensor): mean and log of variance of Q(z|X)

    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_sigma = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_sigma) * epsilon

In [ ]:
def plot_results(models,
                 data,
                 batch_size=128,
                 model_name="vae_mnist"):
    """Plots labels and MNIST digits as a function of the 2D latent vector

    # Arguments
        models (tuple): encoder and decoder models
        data (tuple): test data and label
        batch_size (int): prediction batch size
        model_name (string): which model is using this function
    """

    encoder, decoder = models
    x_test, y_test = data
    os.makedirs(model_name, exist_ok=True)

    filename = os.path.join(model_name, "vae_mean.png")
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = encoder.predict(x_test,
                                   batch_size=batch_size)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=y_test)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.savefig(filename)
    plt.show()

    filename = os.path.join(model_name, "digits_over_latent.png")
    # display a 30x30 2D manifold of digits
    n = 30
    digit_size = 28
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-4, 4, n)
    grid_y = np.linspace(-4, 4, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[i * digit_size: (i + 1) * digit_size,
                   j * digit_size: (j + 1) * digit_size] = digit

    plt.figure(figsize=(10, 10))
    start_range = digit_size // 2
    end_range = (n - 1) * digit_size + start_range + 1
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap='Greys_r')
    plt.savefig(filename)
    plt.show()

In [ ]:
# MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
image_size = x_train.shape[1]
original_dim = image_size * image_size
x_train = np.reshape(x_train, [-1, original_dim])
x_test = np.reshape(x_test, [-1, original_dim])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [ ]:
# network parameters
input_shape = (original_dim, )
intermediate_dim = 512
batch_size = 128
latent_dim = 2
epochs = 20

In [ ]:
# VAE model = encoder + decoder
# build encoder model
inputs = Input(shape=input_shape, name='encoder_input')
x = Dense(intermediate_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

In [ ]:
# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

In [ ]:
# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()
plot_model(encoder, to_file='vae_mlp_encoder.png', show_shapes=True)

In [ ]:
# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = Dense(original_dim, activation='sigmoid')(x)

In [ ]:
# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()
plot_model(decoder, to_file='vae_mlp_decoder.png', show_shapes=True)

In [ ]:
# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')

In [ ]:
models = (encoder, decoder)
data = (x_test, y_test)

reconstruction_loss = mse(inputs, outputs)

reconstruction_loss *= original_dim
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
vae.summary()


In [ ]:
plot_model(vae,
            to_file='vae_mlp.png',
            show_shapes=True)

In [ ]:
vae.fit(x_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, None))
# vae.save_weights('vae_mlp_mnist.h5')

In [ ]:
plot_results(models,
                data,
                batch_size=batch_size,
                model_name="vae_mlp")

# My VAE MLP

## Imports

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras import Model
from tensorflow.keras.losses import binary_crossentropy
import tensorflow as tf
# from tensorflow.keras import objectives
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import os
import pprint

## Load Data

In [ ]:
songs = pd.read_csv("song_list5.csv")

In [ ]:
x_train = songs[[
    "danceability", "energy", "key", "loudness", "mode", "speechiness", 
    "acousticness", "instrumentalness", "liveness", "valence", "tempo", 
    "duration_ms", "time_signature"]].to_numpy()

In [ ]:
x_train.shape

## Build Model

In [ ]:
batch_size = 32
original_dim = 13
intermediate_dim = 32
latent_dim = 4
epochs = 3

In [ ]:
x = Input(shape=(13,))
h = Dense(intermediate_dim, activation='relu')(x)
z_mean = Dense(latent_dim)(h)
z_log_sigma = Dense(latent_dim)(h)

In [ ]:
def sampling(args):
    z_mean, z_log_sigma = args
    epsilon_std = 1.0
    epsilon = K.random_normal(shape=(latent_dim,),
                              mean=0., stddev=epsilon_std)
    return z_mean + K.exp(z_log_sigma) * epsilon

In [ ]:
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])

In [ ]:
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

In [ ]:
# end-to-end autoencoder
vae = Model(x, x_decoded_mean)

# encoder, from inputs to latent space
encoder = Model(x, z_mean)

# generator, from latent space to reconstructed inputs
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(decoder_input)
_x_decoded_mean = decoder_mean(_h_decoded)
generator = Model(decoder_input, _x_decoded_mean)

In [ ]:
def vae_loss(x, x_decoded_mean):
    xent_loss = binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma), axis=-1)
    return xent_loss + kl_loss

vae.compile(optimizer='rmsprop', loss=vae_loss)

## Train Model

In [ ]:
# tf.config.experimental_run_functions_eagerly(True)

In [ ]:
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# x_train = x_train.astype('float32') / 255.
# x_test = x_test.astype('float32') / 255.
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

vae.fit(x_train, x_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size)#,
        # validation_data=(x_test, x_test))

# Manual Neural Network

In [1]:
import pandas as pd
import numpy as np
from numpy import random

In [2]:
songs = pd.read_csv("song_list5.csv")

In [3]:
x_train = songs[[
    "danceability", "energy", "key", "loudness", "mode", "speechiness", 
    "acousticness", "instrumentalness", "liveness", "valence", "tempo", 
    "duration_ms", "time_signature"]].head(5).to_numpy()

In [4]:
x_train.shape

(5, 13)

In [5]:
x_train

array([[ 4.56000e-01,  2.55000e-01,  9.00000e+00, -1.58050e+01,
         1.00000e+00,  4.80000e-02,  9.46000e-01,  1.70000e-01,
         9.51000e-01,  5.32000e-02,  1.16424e+02,  2.53067e+05,
         4.00000e+00],
       [ 5.35000e-01,  8.06000e-01,  7.00000e+00, -1.02890e+01,
         1.00000e+00,  6.42000e-02,  4.36000e-03,  1.91000e-02,
         4.57000e-01,  3.76000e-01,  9.00890e+01,  6.37330e+04,
         4.00000e+00],
       [ 2.94000e-01,  4.82000e-01,  5.00000e+00, -6.40600e+00,
         1.00000e+00,  4.30000e-02,  4.63000e-01,  0.00000e+00,
         3.35000e-01,  2.04000e-01,  1.66693e+02,  2.55280e+05,
         3.00000e+00],
       [ 5.63000e-01,  6.31000e-01,  0.00000e+00, -5.14400e+00,
         1.00000e+00,  3.24000e-02,  6.35000e-02,  8.46000e-06,
         1.63000e-01,  5.40000e-01,  1.15657e+02,  2.54827e+05,
         4.00000e+00],
       [ 3.58000e-01,  6.11000e-01,  2.00000e+00, -9.75200e+00,
         0.00000e+00,  4.54000e-02,  5.15000e-01,  4.68000e-04,
         1.4

In [6]:
# Normalizing the data
x_train = x_train / np.amax(x_train, axis=0)
x_train

array([[8.09946714e-01, 3.16377171e-01, 1.00000000e+00, 3.07251166e+00,
        1.00000000e+00, 7.47663551e-01, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 9.85185185e-02, 6.78477354e-01, 9.84953276e-01,
        1.00000000e+00],
       [9.50266430e-01, 1.00000000e+00, 7.77777778e-01, 2.00019440e+00,
        1.00000000e+00, 1.00000000e+00, 4.60887949e-03, 1.12352941e-01,
        4.80546793e-01, 6.96296296e-01, 5.25006410e-01, 2.48052994e-01,
        1.00000000e+00],
       [5.22202487e-01, 5.98014888e-01, 5.55555556e-01, 1.24533437e+00,
        1.00000000e+00, 6.69781931e-01, 4.89429175e-01, 0.00000000e+00,
        3.52260778e-01, 3.77777778e-01, 9.71427073e-01, 9.93566416e-01,
        7.50000000e-01],
       [1.00000000e+00, 7.82878412e-01, 0.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 5.04672897e-01, 6.71247357e-02, 4.97647059e-05,
        1.71398528e-01, 1.00000000e+00, 6.74007553e-01, 9.91803311e-01,
        1.00000000e+00],
       [6.35879218e-01, 7.58064516e-

In [8]:
class NeuralNetwork: 
    
    def __init__(self):
        # Setup Arch
        self.inputs = 13
        self.encode1 = 64
        self.encode2 = 32

        self.latent = 4

        self.decode1 = 32
        self.decode2 = 64
        self.outputs = 13
        
        # Initialize Weights
        self.weights1 = np.random.randn(self.inputs, self.encode1)
        self.weights2 = np.random.randn(self.encode1, self.encode2)
        self.weights3a = np.random.randn(self.encode2, self.latent)
        self.weights3b = np.random.randn(self.encode2, self.latent)
        self.weights4 = np.random.randn(self.latent, self.decode1)
        self.weights5 = np.random.randn(self.decode1, self.decode2)
        self.weights6 = np.random.randn(self.decode2, self.outputs)

    def sigmoid(self, X):
        return 1 / (1+np.exp(-X))
    def sigmoidPrime(self, X):
        sX = self.sigmoid(X)
        return sX * (1-sX)

    def relu(self, X):
        return np.maximum(0,X)
    def reluPrime(self, X):
        X[X<=0] = 0
        X[X>0] = 1

    def softmax(self, X):
        expo = np.exp(X)
        expo_sum = np.sum(np.exp(X))
        return expo/expo_sum

    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        """
        

        self.hidden_sum1 = np.dot(X, self.weights1)
        self.activated_hidden1 = self.relu(self.hidden_sum1)

        self.hidden_sum2 = np.dot(self.activated_hidden1, self.weights2)
        self.activated_hidden2 = self.relu(self.hidden_sum2)

        self.z_log_sigma = np.dot(self.activated_hidden2, self.weights3a)
        self.z_mean = np.dot(self.activated_hidden2, self.weights3b)

        self.z_sample = self.z_mean + np.exp(self.z_log_sigma) * random.normal(loc=0, scale=1, size=(self.latent, ))
        # random.normal(loc=0, scale=1, size=(4, )) # loc is mean | scale is std | size is shape

        self.hidden_sum4 = np.dot(self.z_sample, self.weights4)
        self.activated_hidden4 = self.relu(self.hidden_sum4)

        self.hidden_sum5 = np.dot(self.activated_hidden4, self.weights5)
        self.activated_hidden5 = self.relu(self.hidden_sum5)

        self.hidden_sum6 = np.dot(self.activated_hidden5, self.weights6)

        return self.hidden_sum6

    def backward(self, X,o):
        """
        Back prop thru the network
        """
        self.mean_square_error = ((o-X) ** 2).mean()
        self.reconstruction_loss = self.mean_square_error * self.inputs
        self.kl_loss = (1+self.z_log_sigma-(self.z_mean**2)-np.exp(self.z_log_sigma)) * -0.5
        self.o_error = (reconstruction_loss + kl_loss).mean()

        self.o_delta = self.o_error * self.reluPrime(self.hidden_sum6)


        # self.weights1 += 
        # self.weights2 += 
        # self.weights3a += 
        # self.weights3b += 
        # self.weights4 += 
        # self.weights5 += 
        # self.weights6 += 











        
        # self.o_error = y - o
        
        # # Apply derivative of sigmoid to error
        # self.o_delta = self.o_error * self.sigmoidPrime(self.output_sum)
        
        # # z2 error: how much were our output layer weights off
        # self.z2_error = self.o_delta.dot(self.weights2.T)
        
        # # z2 delta: how much were the weights off?
        # self.z2_delta = self.z2_error*self.sigmoidPrime(self.hidden_sum)

        # self.weights1 += X.T.dot(self.z2_delta) #Adjust first set (input => hidden) weights
        # self.weights2 += self.activated_hidden.T.dot(self.o_delta) #adjust second set (hidden => output) weights

In [28]:
def reluPrime(X):
    X[X<=0] = 0
    X[X>0] = 1

In [9]:
nn = NeuralNetwork()
output = nn.feed_forward(x_train[0])
print("intput", x_train[0])
print("output", output)

intput [0.80994671 0.31637717 1.         3.07251166 1.         0.74766355
 1.         1.         1.         0.09851852 0.67847735 0.98495328
 1.        ]
output [-8.06226197e+29 -6.83196746e+29 -1.12899827e+29 -3.88896915e+28
 -5.28620813e+29  1.32565895e+29  5.66417669e+29 -7.57264393e+29
  1.09158132e+30 -2.47640496e+29  3.02869135e+29 -4.98384542e+29
  2.18571433e+29]


In [29]:
mean_square_error = ((output-x_train[0]) ** 2).mean()
reconstruction_loss = mean_square_error * nn.inputs
kl_loss = (1+nn.z_log_sigma-(nn.z_mean**2)-np.exp(nn.z_log_sigma)) * -0.5
o_error = (reconstruction_loss + kl_loss).mean()
o_delta = o_error * nn.hidden_sum6
z2_error = o_delta.dot(nn.weights6.T)
hidden_sum5 = nn.hidden_sum5
# z2_delta = 
z2_error * reluPrime(hidden_sum5)


# o_error * reluPrime(self.output_sum)


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

In [26]:
self.z2_delta = self.z2_error*self.sigmoidPrime(self.hidden_sum)

In [ ]:
test_weights1 = np.random.randn(13, 64)
test_weights2 = np.random.randn(64, 32)
test_weights3 = np.random.randn(32, 4)
test_weights4 = np.random.randn(4, 32)
test_weights5 = np.random.randn(32, 64)
test_weights6 = np.random.randn(64, 13)

X1 = np.dot(x_train[0], test_weights1)
X2 = np.maximum(0,X1)
X3 = np.dot(X2, test_weights2)
X4 = np.maximum(0,X3)
X5 = np.dot(X4, test_weights3)
X6 = X5
X7 = np.dot(X6, test_weights4)
X8 = np.maximum(0,X7)
X9 = np.dot(X8, test_weights5)
X10 = np.maximum(0,X9)
X11 = np.dot(X10, test_weights6)
X11

In [ ]:
# Error calculation
mean_square_error = ((output-x_train[0]) ** 2).mean()
reconstruction_loss = mean_square_error * nn.inputs
kl_loss = (1+nn.z_log_sigma-(nn.z_mean**2)-np.exp(nn.z_log_sigma)) * -0.5
vae_loss = (reconstruction_loss + kl_loss).mean()
vae_loss

In [ ]:
# adam optimizer settings
# Good default settings for the tested machine learning problems are alpha=0.001, beta1=0.9, beta2=0.999 and epsilon=10e−8